Create problem

In [23]:
	
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic

In [24]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [25]:
problem = create_problem(10, density=0.15, noise_level=10, negative_values=False)

In [26]:
problem

array([[    0.,    inf, 10573.,    inf,  8430.,    inf,    inf,   831.,
         1977.,    inf],
       [   inf,     0.,    inf,    inf,    inf,  2434.,    inf,    inf,
           inf,  6771.],
       [   inf,    inf,     0.,    inf,    inf,    inf,    inf,    inf,
           inf,  2208.],
       [   inf,    inf,  8563.,     0.,  7768.,    inf,    inf,    inf,
           inf,    inf],
       [ 7330.,    inf,    inf,  8367.,     0.,    inf,    inf,    inf,
           inf,    inf],
       [   inf,    inf,    inf,    inf,    inf,     0.,    inf,    inf,
           inf,    inf],
       [   inf,    inf,    inf,    inf,    inf,    inf,     0.,  5247.,
           inf,    inf],
       [   inf,    inf,  5287.,    inf,    inf,    inf,    inf,     0.,
           inf,    inf],
       [   inf,    inf,    inf, 10443.,  8363.,    inf,  5258.,    inf,
            0.,    inf],
       [   inf,    inf,    inf,    inf,    inf,    inf,  7850.,  7752.,
           inf,     0.]])

In [27]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [28]:
for s, d in combinations(range(problem.shape[0]), 2):
    try:
        path = nx.shortest_path(G, s, d, weight='weight')
        cost = cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    ic(s, d, path, cost)
None

ic| s: 0, d: 1, path: None, cost: inf
ic| s: 0, d: 2, path: [0, 7, 2], cost: 6118.0
ic| s: 0, d: 3, path: [0, 8, 3], cost: 12420.0
ic| s: 0, d: 4, path: [0, 4], cost: 8430.0
ic| s: 0, d: 5, path: None, cost: inf
ic| s: 0, d: 6, path: [0, 8, 6], cost: 7235.0
ic| s: 0, d: 7, path: [0, 7], cost: 831.0
ic| s: 0, d: 8, path: [0, 8], cost: 1977.0
ic| s: 0, d: 9, path: [0, 7, 2, 9], cost: 8326.0
ic| s: 1, d: 2, path: [1, 9, 7, 2], cost: 19810.0
ic| s: 1, d: 3, path: None, cost: inf
ic| s: 1, d: 4, path: None, cost: inf
ic| s: 1, d: 5, path: [1, 5], cost: 2434.0
ic| s: 1, d: 6, path: [1, 9, 6], cost: 14621.0
ic| s: 1, d: 7, path: [1, 9, 7], cost: 14523.0
ic| s: 1, d: 8, path: None, cost: inf
ic| s: 1, d: 9, path: [1, 9], cost: 6771.0
ic| s: 2, d: 3, path: None, cost: inf
ic| s: 2, d: 4, path: None, cost: inf
ic| s: 2, d: 5, path: None, cost: inf
ic| s: 2, d: 6, path: [2, 9, 6], cost: 10058.0
ic| s: 2, d: 7, path: [2, 9, 7], cost: 9960.0
ic| s: 2, d: 8, path: None, cost: inf
ic| s: 2, d: 9, pat

parameter

size = [10, 20, 50, 100, 200, 500, 1000]
density = [0.2, 0.5, 0.8, 1]
noise_level = [0.0, 0.1, 0.5, 0.8]
negative_values = [F, T]


In [29]:
#start code

# > 0 Dijkstra
def solve_instance_dijkstra(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,  
        s: int = int,
        d: int = int  
) -> float:
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed
    )

    if(problem < 0).any():
        print("Error: Negative values in the problem matrix.")

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    try:
        path = nx.shortest_path(G, source = s, target = d, weight='weight')
        cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    return problem, path, cost


In [30]:
# <0 Bellman-Ford
def solve_instance_bellman_ford(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,
        s: int = 0,
        d: int = 1,
):
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed
    )

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    for s, d in combinations(range(problem.shape[0]), 2):
        try:
            path = nx.bellman_ford_path(G, source=s, target=d, weight="weight")
            cost = nx.path_weight(G, path, weight="weight")
            status = "ok"               # path found, no negative cycle
            ic(s, d, path, cost)

        except nx.NetworkXNoPath:
            path = None
            cost = np.inf
            status = "no_path"          # no path found

        except nx.NetworkXUnbounded:
            path = None
            cost = -np.inf
            status = "neg_cycle"        # negative cycle detected

        # cost check
        if status == "ok":
            if cost <= 0:
                status = "not_positive" # total cost <= 0


        if status == "no_path":
            print(f"No path from {s} to {d}")
        elif status == "neg_cycle":
            print(f"There is a negative cycle reachable from {s} that can affect {d}; shortest path is undefined. cost = {cost}")
        elif status == "not_positive":
            print(f"Path exists but cost = {cost} (not positive, so we discard it).")
        else:  
            print(f"Shortest path from {s} to {d}: {path} with cost {cost}")
    return problem, path, cost, status


In [31]:
import random
size = 10
density = 0.5
noise_level = 0.5
negative_values = True
seed = random.randint(0, 100)
s = 0
d = size - 1

problem, path, cost, status = solve_instance_bellman_ford(
    size=size,
    density=density,
    noise_level=noise_level,
    negative_values=negative_values,
    seed=seed,
    s=s,
    d=d,
)

print("Problem Matrix:\n", problem)


Shortest path from 0 to 1: [0, 2, 4, 1] with cost 594.0
Shortest path from 0 to 2: [0, 2] with cost 538.0
Shortest path from 0 to 3: [0, 3] with cost 452.0
Shortest path from 0 to 4: [0, 2, 4] with cost 435.0
Shortest path from 0 to 5: [0, 5] with cost 647.0
Shortest path from 0 to 6: [0, 3, 6] with cost 263.0
Shortest path from 0 to 7: [0, 7] with cost 1031.0
Shortest path from 0 to 8: [0, 3, 8] with cost 854.0
Shortest path from 0 to 9: [0, 3, 9] with cost 559.0
Shortest path from 1 to 2: [1, 5, 2] with cost 912.0
Shortest path from 1 to 3: [1, 3] with cost 524.0
Shortest path from 1 to 4: [1, 5, 2, 4] with cost 809.0
Shortest path from 1 to 5: [1, 5] with cost 717.0
Shortest path from 1 to 6: [1, 3, 6] with cost 335.0
Shortest path from 1 to 7: [1, 7] with cost 705.0
Shortest path from 1 to 8: [1, 3, 8] with cost 926.0
Shortest path from 1 to 9: [1, 9] with cost 533.0
Shortest path from 2 to 3: [2, 8, 3] with cost 182.0
Path exists but cost = -103.0 (not positive, so we discard it).

ic| s: 0, d: 1, path: [0, 2, 4, 1], cost: 594.0
ic| s: 0, d: 2, path: [0, 2], cost: 538.0
ic| s: 0, d: 3, path: [0, 3], cost: 452.0
ic| s: 0, d: 4, path: [0, 2, 4], cost: 435.0
ic| s: 0, d: 5, path: [0, 5], cost: 647.0
ic| s: 0, d: 6, path: [0, 3, 6], cost: 263.0
ic| s: 0, d: 7, path: [0, 7], cost: 1031.0
ic| s: 0, d: 8, path: [0, 3, 8], cost: 854.0
ic| s: 0, d: 9, path: [0, 3, 9], cost: 559.0
ic| s: 1, d: 2, path: [1, 5, 2], cost: 912.0
ic| s: 1, d: 3, path: [1, 3], cost: 524.0
ic| s: 1, d: 4, path: [1, 5, 2, 4], cost: 809.0
ic| s: 1, d: 5, path: [1, 5], cost: 717.0
ic| s: 1, d: 6, path: [1, 3, 6], cost: 335.0
ic| s: 1, d: 7, path: [1, 7], cost: 705.0
ic| s: 1, d: 8, path: [1, 3, 8], cost: 926.0
ic| s: 1, d: 9, path: [1, 9], cost: 533.0
ic| s: 2, d: 3, path: [2, 8, 3], cost: 182.0
ic| s: 2, d: 4, path: [2, 4], cost: -103.0
ic| s: 2, d: 5, path: [2, 4, 5], cost: 162.0
ic| s: 2, d: 6, path: [2, 8, 3, 6], cost: -7.0
ic| s: 2, d: 7, path: [2, 8, 7], cost: 678.0
ic| s: 2, d: 8, path: [2, 8

In [32]:
import numpy as np
import networkx as nx
from itertools import product

# get a copy of the coordinates used in create_problem
def get_coords(size: int, seed: int = 42) -> np.ndarray:
    rng = np.random.default_rng(seed)
    coords = rng.random(size=(size, 2))  # same as in create_problem
    return coords

In [51]:
def solve_instance_astar_test(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,
        s: int = 0,
        d: int = 1,
):
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed
    )

    #for h(n)
    coords = get_coords(size, seed=seed)

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

    # heuristic function,when negative_values=True, it's not admissible,is only offering guidance,not try to find optimal path
    def heuristic(u: int, v: int) -> float:
        dx = coords[u, 0] - coords[v, 0]
        dy = coords[u, 1] - coords[v, 1]
        return np.sqrt(dx * dx + dy * dy) * 1000


    # a*
    for  s, d in combinations(range(problem.shape[0]), 2):
        try:
            path = nx.astar_path(G, source=s, target=d,
                                heuristic=heuristic, weight="weight")
            cost = nx.path_weight(G, path, weight="weight")
            ic(s, d, path, cost)
        except nx.NetworkXNoPath:
            path = None
            cost = np.inf

        #  check if there is any negative edge in the path
        has_negative_edge = False
        negative_edges = []
        if path is not None and len(path) > 1:
            for i in range(len(path) - 1):
                u, v = path[i], path[i + 1]
                w = problem[u, v]
                if w < 0:
                    has_negative_edge = True
                    negative_edges.append((u, v, w))
        if path is None:
            print(f"A* Can't find a path from {s} to {d}.")
        else:
            print(f"A* path: {path}")
            print(f"total cost: {cost}")
            print("Is there any neg_value path:", has_neg)
        if has_neg:
            print("neg_values edge(s) is/are：")
            for s, d, w in neg_edges:
                print(f"  {s} -> {d}: {w}")

    return problem, path, cost, has_negative_edge, negative_edges


In [ ]:
size = 10
density = 0.5
noise_level = 0.5
negative_values = True   
seed = random.randint(0, 100)
s = 0
d = size - 1

problem, path, cost, has_neg, neg_edges = solve_instance_astar_test(
    size=size,
    density=density,
    noise_level=noise_level,
    negative_values=negative_values,
    seed=seed,
    s=s,
    d=d,
)

#print("Problem Matrix:\n", problem)

ic(problem)


: 